In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0 - Importing Packages

In [85]:
import matplotlib.pyplot as plt  #Visualization
import plotly.express as px #Visualization 
#!pip install pmdarima
from pmdarima import auto_arima #Predictions
import plotly.graph_objs as go

from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# 1 - Data Exploration

Let's study Apple stocks.

In [86]:
df = pd.read_csv('../input/stock-market-dataset/stocks/AAPL.csv') 
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.513393,0.515625,0.513393,0.513393,0.406782,117258400
1,1980-12-15,0.488839,0.488839,0.486607,0.486607,0.385558,43971200
2,1980-12-16,0.453125,0.453125,0.450893,0.450893,0.357260,26432000
3,1980-12-17,0.462054,0.464286,0.462054,0.462054,0.366103,21610400
4,1980-12-18,0.475446,0.477679,0.475446,0.475446,0.376715,18362400
...,...,...,...,...,...,...,...
9904,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
9905,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
9906,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
9907,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500


In [87]:
df.set_index('Date',drop=False, inplace=True)
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-03-26,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
2020-03-27,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
2020-03-30,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
2020-03-31,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500
2020-04-01,2020-04-01,246.500000,248.720001,239.130005,240.910004,240.910004,43956200


Let's see a brief description of the dataset

In [88]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,9909.000000,9909.000000,9909.000000,9909.000000,9909.000000,9.909000e+03
mean,32.606849,32.936079,32.277560,32.618030,30.576570,8.582916e+07
std,58.415759,59.001576,57.883037,58.471899,56.746275,8.597195e+07
min,0.198661,0.198661,0.196429,0.196429,0.155638,3.472000e+05
25%,1.071429,1.089286,1.048571,1.071429,0.917643,3.304230e+07
50%,1.729286,1.758929,1.696429,1.732143,1.466154,5.766490e+07
75%,35.799999,36.265713,35.328571,35.761429,31.042374,1.069992e+08
max,324.739990,327.850006,323.350006,327.200012,327.200012,1.855410e+09


In [89]:
fig_open = px.line(data_frame=df, x='Date', y=['Open'])

fig_open.update_yaxes( # the y-axis is in dollars
    tickprefix="$", showgrid=False, title='Open Price')
fig_open.update_xaxes(
    title = 'Date')

fig_open.show()

In [90]:
fig_close = px.line(data_frame=df, x='Date', y=['Close'])

fig_close.update_yaxes( # the y-axis is in dollars
    tickprefix="$", showgrid=False, title='Close Price')
fig_close.update_xaxes(
    title = 'Date')

fig_close.show()

Since this notebook is just for learning, let's use the data since 2015.

In [91]:
df_lastyears = df.loc['2015-01-02':]
df_lastyears.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2020-03-26,2020-03-26,246.520004,258.679993,246.360001,258.440002,258.440002,63021800
2020-03-27,2020-03-27,252.750000,255.869995,247.050003,247.740005,247.740005,51054200
2020-03-30,2020-03-30,250.740005,255.520004,249.399994,254.809998,254.809998,41994100
2020-03-31,2020-03-31,255.600006,262.489990,252.000000,254.289993,254.289993,49250500
2020-04-01,2020-04-01,246.500000,248.720001,239.130005,240.910004,240.910004,43956200


In [92]:
fig_close_2 = px.line(data_frame=df_lastyears, x='Date', y=['Close'])

fig_close_2.update_yaxes(tickprefix="$", showgrid=False, title='Close Price')
fig_close_2.update_xaxes(title = 'Date')

fig_close_2.show()

In [93]:
fig_open_2 = px.line(data_frame=df, x='Date', y=['Open'])

fig_open_2.update_yaxes(tickprefix="$", showgrid=False, title='Open Price')
fig_open_2.update_xaxes(title = 'Date')

fig_open_2.show()

In [94]:
Fig = go.Figure([
                 go.Scatter(
                     name = 'Close',
                     x = df_lastyears.index,
                     y = df_lastyears['Close'],
                     mode= 'markers'
                 ),
                 go.Scatter(
                     name = 'Open',
                     x = df_lastyears.index,
                     y = df_lastyears['Open'],
                     mode= 'markers'
                 )

])

Fig.update_layout(
    yaxis_title='Price',
    yaxis_tickprefix='$',
    title = 'Precios de apertura y cierre'
)
Fig.update_traces(
    marker = dict(size=2.5)
)

Fig.show()

# 2 - Feature Engineering

Let's calculate daily return by using the following expression: <h3>
$Return[i] = \frac{Close[i-1]-Close[i]}{Close[i]}$

In [95]:
#Price difference between two consecutive days
df_lastyears['Price_Difference'] = df_lastyears['Close'].shift(-1) - df_lastyears['Close']
df_lastyears = df_lastyears.dropna()

#Let's calculate the daily return by using the expression we wrote before
df_lastyears['Return'] = df_lastyears['Price_Difference'] / df_lastyears['Close']

fig_return = px.line(data_frame=df_lastyears, x=df_lastyears.index, y='Return', title='Daily Return')
fig_return.update_xaxes(title='Date')
fig_return.show()

Now, lets calculate some moving averages:

In [96]:
df_lastyears['MA10_Close'] = df_lastyears['Close'].rolling(10).mean()
df_lastyears['MA10_Open'] = df_lastyears['Open'].rolling(10).mean()

df_lastyears['MA50_Close'] = df_lastyears['Close'].rolling(50).mean()
df_lastyears['MA50_Open'] = df_lastyears['Open'].rolling(50).mean()

In [97]:
MA_Fig = go.Figure([
                    
                    go.Scatter(
                        name='MA10 Close',
                        x = df_lastyears.index,
                        y = df_lastyears['MA10_Close']

                    ),
                    go.Scatter(
                        name='MA50 Close',
                        x = df_lastyears.index,
                        y = df_lastyears['MA50_Close']

                    ),
                    go.Scatter(
                        name='Close Price',
                        x = df_lastyears.index,
                        y = df_lastyears['Close'],
                        mode = 'markers'

                    )
])
MA_Fig.update_layout(
    yaxis_title = 'Price',
    title = 'Close Price & MA10, MA50'
)

MA_Fig.update_traces(
    marker = dict(size=4)
)

MA_Fig.show()

# 4 - Price Predictions

Let's separate our data in train and test.

In [98]:
train_data, test_data = df_lastyears[0:int(len(df_lastyears)*0.7)], df_lastyears[int(len(df_lastyears)*0.7):]

In [99]:
train_data

,Date,Open,High,Low,Close,Adj Close,Volume,Price_Difference,Return,MA10_Close,MA10_Open,MA50_Close,MA50_Open
Date,,,,,,,,,,,,,
2015-01-02,2015-01-02,111.389999,111.440002,107.349998,109.330002,100.216454,53204600,-3.080002,-0.028172,NaN,NaN,NaN,NaN
2015-01-05,2015-01-05,108.290001,108.650002,105.410004,106.250000,97.393181,64285500,0.010002,0.000094,NaN,NaN,NaN,NaN
2015-01-06,2015-01-06,106.540001,107.430000,104.629997,106.260002,97.402374,65797100,1.489998,0.014022,NaN,NaN,NaN,NaN
2015-01-07,2015-01-07,107.199997,108.199997,106.699997,107.750000,98.768150,40105900,4.139999,0.038422,NaN,NaN,NaN,NaN
2015-01-08,2015-01-08,109.230003,112.150002,108.699997,111.889999,102.563072,59364500,0.120003,0.001073,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-24,2018-08-24,216.600006,216.899994,215.110001,216.160004,211.717331,18476400,1.779999,0.008235,213.696002,213.413002,196.634801,196.400601
2018-08-27,2018-08-27,217.149994,218.740005,216.330002,217.940002,213.460739,20525100,1.759995,0.008076,214.603003,214.197002,197.216801,196.943000
2018-08-28,2018-08-28,219.009995,220.539993,218.919998,219.699997,215.184570,22776800,3.279999,0.014929,215.598003,215.082001,197.836001,197.565600


In [100]:
Fig_tt = go.Figure([
                    
                    go.Scatter(
                        name= 'Train closes',
                        x = train_data.index,
                        y = train_data['Close'],
                        mode = 'markers'
                    ),
                    
                    go.Scatter(
                        name = 'Test closes',
                        x = test_data.index, 
                        y = test_data['Close'],
                        mode = 'markers'

                    )
])

Fig.update_layout(
    yaxis_title = 'Price ($)',
    title = 'Training & Test Close Prices'
)
Fig_tt.show()

Let's configure our model

In [ ]:
ex_variables = ['MA50_Close', 'MA50_Open', 'MA10_Open', 'MA10_Close']
train_data = train_data.dropna()
model = auto_arima(
    train_data['Close'],
    exogenous=train_data[ex_variables],
    trace=True, error_action="ignore",
    suppress_warnings=True)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=3.30 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=4061.614, Time=0.88 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=3173.395, Time=1.17 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=3483.401, Time=1.20 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=4071.481, Time=0.50 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=3167.091, Time=1.36 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=3160.508, Time=3.23 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=3150.189, Time=3.62 sec
 ARIMA(5,0,0)(0,0,0)[0] intercept   : AIC=3154.305, Time=4.57 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept   : AIC=3140.209, Time=4.02 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept   : AIC=3169.313, Time=3.00 sec
 ARIMA(5,0,1)(0,0,0)[0] intercept   : AIC=3157.349, Time=4.15 sec
 ARIMA(4,0,2)(0,0,0)[0] intercept   : AIC=3175.866, Time=3.86 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept   : AIC=3182.904, Time=3.04 sec
 ARIMA(5,0,2)(0,0,0)[0] intercept   : 

Now let's fit our model

In [ ]:
model.fit(train_data['Close'], exogenous=train_data[ex_variables])


Let's drop any $NaN$ value that we could have:

In [ ]:
test_data = test_data.dropna()

And now let's make predictions

In [ ]:
predictions = model.predict(n_periods=len(test_data), exogenous=test_data[ex_variables])
test_data['Predictions'] = predictions

In [ ]:
pred_Fig  = go.Figure([
                       
                       go.Scatter(
                        name= 'Train closes',
                        x = train_data.index,
                        y = train_data['Close'],
                        mode = 'markers'
                    ),
                    
                    go.Scatter(
                        name = 'Test closes',
                        x = test_data.index, 
                        y = test_data['Close'],
                        mode = 'markers'

                    ),

                    go.Scatter(
                        name = 'Predictions',
                        x = test_data.index,
                        y = test_data['Predictions'],
                        mode = 'lines'
                    )
])

pred_Fig.show()